In [5]:
# Crawling에 필요한 module 선언
import time

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# (201105) 추가한 모듈
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup

# lxml
from lxml import html
import xml.etree.ElementTree as ET

# pandas
import pandas as pd

# requests, json, re
import requests
import json
import re

In [22]:
# 유튜브 정식 카테고리 k, v dictionary
youtube_category_dict = {
    'Film & Animation' : 1,
    'Autos & Vehicles' : 2,
    'Music' : 10,
    'Pets & Animals' : 15,
    'Sports' : 17,
    'Short Movies' : 18,
    'Travel & Events' : 19,
    'Gaming' : 20,
    'Videoblogging' : 21,
    'People & Blogs' : 22,
    'Comedy' : 23,
    'Entertainment' : 24,
    'News & Politics' : 25,
    'Howto & Style' : 26,
    'Education' : 27,
    'Science & Technology' : 28,
    'Movies' : 30,
    'Anime/Animation' : 31,
    'Action/Adventure' : 32,
    'Classics' : 33,
    'Documentary' : 35,
    'Drama' : 36,
    'Family' : 37,
    'Foreign' : 38,
    'Horror' : 39,
    'Sci-Fi/Fantasy' : 40,
    'Thriller' : 41,
    'Shorts' : 42,
    'Shows' : 43,
    'Trailers' :44
}

# 샘플 키워드 리스트
sports_list = ['한국 프로야구', '해외축구', '바르셀로나', '배구', '손흥민']
movie_list = ['영화리뷰', '영화 명장면', '명대사', '한국 영화']
kids_list = ['뽀로로', '헤이지니', '카봇', '보람튜브', '핑크퐁', '공룡']
music_list = ['인기 음악', 'k-pop', '팝송', '카페에서 듣기좋은 노래', '호른', '클래식', '째즈']
comedy_list = ['런닝맨', '1박2일 다시보기', '무한도전', '놀면 뭐하니', 'tvN']
game_list = ['롤', '피파', 'fps', '배그', '서든어택', '어몽어스']
politics_list = ['바이든', '트럼프', '대통령', '중국']
cook_list = ['백종원', '자취음식', '편의점 음식만들기', '몸에 좋은 음식']
animal_list = ['고양이 영상', '비글', '해수어', '상어']

# 모든 키워드를 다 합친 리스트
all_search_keyword_list = sports_list + movie_list + kids_list + music_list + comedy_list + game_list + politics_list + cook_list + animal_list

print("All keywords list: ", all_search_keyword_list)

# video_id_list
# sports_video_id_list = []
# movie_video_id_list = []
# kids_video_id_list = []
# music_video_id_list = []
# comedy_video_id_list = []
# game_video_id_list = []
# politics_video_id_list = []
# cook_video_id_list = []
# animal_video_id_list = []

# 샘플 키워드 딕셔너리
# search_dict = {
#     'Sports' : [sports_list, sports_video_id_list]
#     'Movie' : [movie_list, movie_video_id_list]
#     'Kids' : [kids_list, kids_video_id_list]
#     'Music' : [music_list, music_video_id_list]
#     'Comedy' : [comedy_list, comedy_video_id_list]
#     'Game' : [game_list, game_video_id_list]
#     'Politics' : [politics_list, politics_video_id_list]
#     'Cook' : [cook_list, cook_video_id_list]
#     'Animal' : [animal_list, animal_video_id_list]
# }

All keywords list:  ['한국 프로야구', '해외축구', '바르셀로나', '배구', '손흥민', '영화리뷰', '영화 명장면', '명대사', '한국 영화', '뽀로로', '헤이지니', '카봇', '보람튜브', '핑크퐁', '공룡', '인기 음악', 'k-pop', '팝송', '카페에서 듣기좋은 노래', '호른', '클래식', '째즈', '런닝맨', '1박2일 다시보기', '무한도전', '놀면 뭐하니', 'tvN', '롤', '피파', 'fps', '배그', '서든어택', '어몽어스', '바이든', '트럼프', '대통령', '중국', '백종원', '자취음식', '편의점 음식만들기', '몸에 좋은 음식', '고양이 영상', '비글', '해수어', '상어']


In [35]:
# 검색어 결과별로 비디오 id를 모아주는 함수 정의
# 리턴은 dictionary
def get_search_keyword_scrap_video_id(search_keyword_list):
    # 최종적으로 리턴할 dictionary 정의
    youtube_id_dict = {}

    for word in search_keyword_list:
        # youtube_id_list
        youtube_id_list = []
        
        # webdriver define
        driver_path = './chromedriver'
        driver = webdriver.Chrome(driver_path)
        driver.implicitly_wait(30) # or bigger second
        
        # 열고자 하는 search url 정의
        url = "https://www.youtube.com/results?search_query=" + word
        print("full url: ", url)

        # 해당 url로 창 열기
        driver.get(url)
        time.sleep(5)

        # 첫 동영상 시작 시 시작번호(index = 1)
        index = 1
        SCROLL_PAUSE_TIME = 2

        # infinite scroll
        while True:
            try:
                youtube_video_id_xpath = '/html/body/ytd-app/div/ytd-page-manager/ytd-search/div[1]/ytd-two-column-search-results-renderer/div/ytd-section-list-renderer/div[2]/ytd-item-section-renderer[1]/div[3]/ytd-video-renderer['+str(index)+']/div[1]/div/div[1]/div/h3/a'

                # 해당 페이지의 8개의 영상을 모두 확인했다면 그다음으로 가기위해 스크롤
                if index % 4 == 0:
                    # 2초 대기 (대기 시간은 크롤러의 정밀도에 따라 조정 가능)
                    # driver.execute_script("window.scrollBy(0, 1080);")
                    # time.sleep(SCROLL_PAUSE_TIME)
                    # driver.execute_script("window.scrollBy(0, 1080);")
                    # time.sleep(SCROLL_PAUSE_TIME)
                    # driver.execute_script("window.scrollBy(0, 1080);")
                    # time.sleep(SCROLL_PAUSE_TIME)
                    last_height = driver.execute_script("return document.body.scrollHeight")

                    while True:
                        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
                        time.sleep(SCROLL_PAUSE_TIME)

                        new_height = driver.execute_script("return document.body.scrollHeight")

                        if new_height == last_height:
                            break

                        last_height = new_height


                youtube_title_link = driver.find_element_by_xpath(youtube_video_id_xpath)
                youtube_id = youtube_title_link.get_attribute('href')
                youtube_id = youtube_id.split("v=")[1].split("&")[0]
                youtube_id_list.append(youtube_id)

                # 리스트에 들어간 데이터 중간 확인
                print("index_no: ", index)
                print("youtube_id: ", youtube_id)

                index += 1

            except Exception as e:
                print()
                print("Exception: ", e)
                break
        # dictionary에 검색 키워드 별 리스트 추가
        youtube_id_dict[word] = youtube_id_list
        # youtube_id 수집이 끝난 브라우저 닫기        
        driver.close()
    
    return youtube_id_dict

In [36]:
youtube_id_dict = get_search_keyword_scrap_video_id(all_search_keyword_list)
print("youtube_id_dict: ", youtube_id_dict)

full url:  https://www.youtube.com/results?search_query=한국 프로야구
index_no:  1
youtube_id:  doorlUyE9pI
index_no:  2
youtube_id:  L0Y38pQbEeQ
index_no:  3
youtube_id:  VjwJSj0AW4g
index_no:  4
youtube_id:  A5vCPtjOe8Q
index_no:  5
youtube_id:  Zeja7fC1Okc
index_no:  6
youtube_id:  7ARbpqOgHM8
index_no:  7
youtube_id:  WC2MB_lZ0HQ
index_no:  8
youtube_id:  ugsFwK74KDU
index_no:  9
youtube_id:  FeqhJVzlE-4
index_no:  10
youtube_id:  B-ZFW6-FBJ8
index_no:  11
youtube_id:  kH-ht4CyUKg
index_no:  12
youtube_id:  kQxq3PNE0gw
index_no:  13
youtube_id:  leBGyyECGPk
index_no:  14
youtube_id:  7eYu507yrQ8
index_no:  15
youtube_id:  -P68flOonvE
index_no:  16
youtube_id:  FCEPBiUPPho
index_no:  17
youtube_id:  pp0a2SbPwRA
index_no:  18
youtube_id:  VFufiK95fGU
index_no:  19
youtube_id:  De58PXoH4IU
index_no:  20
youtube_id:  wVo7-0zDNps

Exception:  Message: no such element: Unable to locate element: {"method":"xpath","selector":"/html/body/ytd-app/div/ytd-page-manager/ytd-search/div[1]/ytd-two-colu

full url:  https://www.youtube.com/results?search_query=명대사
index_no:  1
youtube_id:  ojttyD8aMR0
index_no:  2
youtube_id:  hdU_5M9U9tk
index_no:  3
youtube_id:  0ZHqB7Fplu0
index_no:  4
youtube_id:  goGst1xKBQ4
index_no:  5
youtube_id:  LJO7-pN9hAk
index_no:  6
youtube_id:  EwERszcjwaE
index_no:  7
youtube_id:  F4SUPzLdbu0
index_no:  8
youtube_id:  zm8Ts0XoErg
index_no:  9
youtube_id:  y58EHjtKnBA
index_no:  10
youtube_id:  Q9oat_q3UqQ
index_no:  11
youtube_id:  9fCqGqqrNVk
index_no:  12
youtube_id:  SQfv__xer3A
index_no:  13
youtube_id:  OQVzaW0oBgE
index_no:  14
youtube_id:  XJkG4Kiykm8
index_no:  15
youtube_id:  pYCtl0CEv5w
index_no:  16
youtube_id:  B2mOlO1ERtw
index_no:  17
youtube_id:  _ZWKRJY5Ghs
index_no:  18
youtube_id:  8nRNGweY46g
index_no:  19
youtube_id:  R5NWIGAhHYA
index_no:  20
youtube_id:  dxusUW9_xZE

Exception:  Message: no such element: Unable to locate element: {"method":"xpath","selector":"/html/body/ytd-app/div/ytd-page-manager/ytd-search/div[1]/ytd-two-column-s

full url:  https://www.youtube.com/results?search_query=인기 음악
index_no:  1
youtube_id:  zjGzPiYGhhY
index_no:  2
youtube_id:  khu6-Cdn8E4
index_no:  3
youtube_id:  848a1kMwRpE
index_no:  4
youtube_id:  _BVqb9vWZ_k
index_no:  5
youtube_id:  wMSh6wFNtuo
index_no:  6
youtube_id:  Rum2524qZPU
index_no:  7
youtube_id:  B8FyB1vf5lY
index_no:  8
youtube_id:  YyeylfmSVP8
index_no:  9
youtube_id:  vpbR3NEJmJM
index_no:  10
youtube_id:  Z2_ygVUxUcc
index_no:  11
youtube_id:  USgbeoynpw4
index_no:  12
youtube_id:  0lI5xI_gIIs
index_no:  13
youtube_id:  u-E92ep1BDg
index_no:  14
youtube_id:  -kkyXf_D_TE
index_no:  15
youtube_id:  yJuLOl1V-lQ
index_no:  16
youtube_id:  iHfqD2RYT2I
index_no:  17
youtube_id:  b5rrGmpZlDY
index_no:  18
youtube_id:  JzziwpDrigs
index_no:  19
youtube_id:  N5Nq3xBs5U8

Exception:  Message: no such element: Unable to locate element: {"method":"xpath","selector":"/html/body/ytd-app/div/ytd-page-manager/ytd-search/div[1]/ytd-two-column-search-results-renderer/div/ytd-sectio

full url:  https://www.youtube.com/results?search_query=tvN
index_no:  1
youtube_id:  hc5petZeyio
index_no:  2
youtube_id:  wLCmXq2_yRo

Exception:  Message: no such element: Unable to locate element: {"method":"xpath","selector":"/html/body/ytd-app/div/ytd-page-manager/ytd-search/div[1]/ytd-two-column-search-results-renderer/div/ytd-section-list-renderer/div[2]/ytd-item-section-renderer[1]/div[3]/ytd-video-renderer[3]/div[1]/div/div[1]/div/h3/a"}
  (Session info: chrome=87.0.4280.66)

full url:  https://www.youtube.com/results?search_query=롤
index_no:  1
youtube_id:  lZX6rR2hxmU
index_no:  2
youtube_id:  X96TunaR3Bo
index_no:  3
youtube_id:  87M8if6zrrU
index_no:  4
youtube_id:  sKQIFWIXyMY
index_no:  5
youtube_id:  oaIV0s5DXXI
index_no:  6
youtube_id:  0ayv0_ujL0k
index_no:  7
youtube_id:  1PqhVpGai1c
index_no:  8
youtube_id:  ErhShv4ELL4
index_no:  9
youtube_id:  w_VbC-krBM0
index_no:  10
youtube_id:  1ArtO40izV0
index_no:  11
youtube_id:  W-4f0j-CVf4
index_no:  12
youtube_id:  MugU

index_no:  4
youtube_id:  f_5UCufu9xc
index_no:  5
youtube_id:  e96IeaSYC5I
index_no:  6
youtube_id:  9-_FOYbW--U
index_no:  7
youtube_id:  376iFtKv-7o
index_no:  8
youtube_id:  z1iXjQDwLxU
index_no:  9
youtube_id:  750xpOHhSnI
index_no:  10
youtube_id:  vg_LETfsHu4
index_no:  11
youtube_id:  MS7zuIWmu0Q
index_no:  12
youtube_id:  5Cv7jPp2_iM
index_no:  13
youtube_id:  Y3nxMnZ42b8
index_no:  14
youtube_id:  sYd7qtzzyaQ
index_no:  15
youtube_id:  hgY5kndWD1M
index_no:  16
youtube_id:  nH8bFg6ijqE
index_no:  17
youtube_id:  QL6dTGwUkzI
index_no:  18
youtube_id:  LQElZ4N89Hk
index_no:  19
youtube_id:  g-KxX5XGu5U
index_no:  20
youtube_id:  D3OReBCp8eY

Exception:  Message: no such element: Unable to locate element: {"method":"xpath","selector":"/html/body/ytd-app/div/ytd-page-manager/ytd-search/div[1]/ytd-two-column-search-results-renderer/div/ytd-section-list-renderer/div[2]/ytd-item-section-renderer[1]/div[3]/ytd-video-renderer[21]/div[1]/div/div[1]/div/h3/a"}
  (Session info: chrome=87

index_no:  4
youtube_id:  lZJFhEStBS8
index_no:  5
youtube_id:  t9P4k27dogk
index_no:  6
youtube_id:  nWMMbZG9aKM
index_no:  7
youtube_id:  OWOPZApMOqE
index_no:  8
youtube_id:  rbz5siyFS9s
index_no:  9
youtube_id:  uZFdYhz4ZFU
index_no:  10
youtube_id:  bwQ_jw4lZ3g
index_no:  11
youtube_id:  D99LZeZbdyk
index_no:  12
youtube_id:  QEZzngmPzlQ
index_no:  13
youtube_id:  LGv38Ny4RKc
index_no:  14
youtube_id:  G8Szvw2nGa4
index_no:  15
youtube_id:  RM4rK3vHa_c
index_no:  16
youtube_id:  435bSRyWoAo
index_no:  17
youtube_id:  D52kRtlntGI
index_no:  18
youtube_id:  2vBU7H6OZgw
index_no:  19
youtube_id:  QRFqphsEZeU
index_no:  20
youtube_id:  J3vLTifqG7U

Exception:  Message: no such element: Unable to locate element: {"method":"xpath","selector":"/html/body/ytd-app/div/ytd-page-manager/ytd-search/div[1]/ytd-two-column-search-results-renderer/div/ytd-section-list-renderer/div[2]/ytd-item-section-renderer[1]/div[3]/ytd-video-renderer[21]/div[1]/div/div[1]/div/h3/a"}
  (Session info: chrome=87

In [76]:
# 특정 검색어 키워드 기반으로 크롤링 해 주는 크롤러 정의
def scrap_search_detail_video_data(video_id_dict):
    # 모든 결과를 누적해서 보여주는 df 정의
    total_df = pd.DataFrame()
    
    # 각 video_id에 해당하는 유튜브 영상 별로 상세 데이터 수집
    for keyword, video_id_list in video_id_dict.items():
        for video_id in video_id_list:
            # webdriver 초기화
            driver_path = './chromedriver'
            driver = webdriver.Chrome(driver_path)
            driver.implicitly_wait(60) # or bigger second

            # origin_url 정의
            origin_url = 'https://www.youtube.com/watch?v='

            # 테스트용 단일 youtube_id 정의
            # (임시) 딘일로 넣어주는 값
            # video_id = 'y57Zldv2bik'

            # youtube_detail_raw_data를 받을 dictionary 정의
            youtube_raw_data = {}

            # 넣어주는 url 값
            url = origin_url + video_id

            # scriptTag에서 긁어지는 데이터
            script_tag_xpath = '/html/body/ytd-app/div/ytd-page-manager/ytd-watch-flexy/div[1]/ytd-player-microformat-renderer/script'

            # meta tag에서 긁어지는 데이터 -> soup.find, soup.findAll 이용해서 긁기
            request = requests.get(url)
            soup = BeautifulSoup(request.text, 'lxml')

            # ytInitialData에서 긁어지는 데이터 -> 복잡한 계층의 json 데이터 기반
            yt_initial_data_xpath = '/html/body/script[19]'

            # 수집할 데이터의 리스트
            title_list = []
            author_list = []
            description_list = []
            url_list = []
            thumb_url_list = []
            like_cnt_list = []
            dislike_cnt_list = []
            follow_cnt_list = []
            view_cnt_list = []
            duration_list = []
            youtube_category_list = []
            youtube_category_sn_list = []
            is_family_friendly_list = []
            title_hash_tags_list = []
            description_hash_tags_list = []
            video_id_list = [] 
            author_id_list = []
            yupload_time_list = []
            keywords_list = []

            driver.get(url)
            time.sleep(5)
            
            try:
                # Data 크롤링
                # 각 유형별로 다른 로직으로 데이터를 크롤링한다

                # script tag id=scriptTag의 내부 HTML 내용을 가져온다(scriptTag data crawling)
                script_tag_str = driver.find_element_by_xpath(script_tag_xpath).get_attribute("innerHTML")

                script_tag_json = json.loads(script_tag_str)

                time.sleep(5)

                # print("script_tag_json: ", script_tag_json)
                # print("script_tag_json_type: ", type(script_tag_json))

                # Data result test
                title = script_tag_json['name']
                author = script_tag_json['author']
                description = script_tag_json['description']
                thumb_url = script_tag_json['thumbnailUrl'][0]
                view_cnt = int(script_tag_json['interactionCount'])
                duration = int(script_tag_json['duration'][2:len(script_tag_json['duration'])-1].replace(",", ""))
                youtube_category = script_tag_json['genre']
                # addition : youtube_category_sn
                youtube_category_sn = youtube_category_dict[youtube_category]
                
                print()
                print("=" * 20, "Youtube Raw Data Descriptions", "=" * 20)
                print("title: ", title)
                print("author: ", author)
                print("description: ", description)
                print("thumb_url: ", thumb_url)
                print("view_cnt: ", view_cnt, " type: ", type(view_cnt))
                print("duration: ", duration, " type: ", type(duration))
                print("youtube_category: ", youtube_category)
                print("youtube_category_id: ", youtube_category_sn)

                # scriptTag data appending
                title_list.append(title)
                author_list.append(author)
                description_list.append(description)
                url_list.append(url)
                thumb_url_list.append(thumb_url)
                view_cnt_list.append(view_cnt)
                duration_list.append(duration)
                youtube_category_list.append(youtube_category)
                youtube_category_sn_list.append(youtube_category_sn)

                
                # meta tag data crawling
                is_family_friendly = soup.find("meta", {"itemprop" : "isFamilyFriendly"})['content']
                author_id = soup.find("meta", {"itemprop" : "channelId"})['content']
                yupload_time = soup.find("meta", {"itemprop" : "uploadDate"})['content']
                keywords = ",".join([tag['content'].replace("#", "") for tag in soup.find_all("meta", {"property" : "og:video:tag"})])

                print("is_family_friendly: ", is_family_friendly)
                print("video_id: ", video_id)
                print("author_id: ", author_id)
                print("yupload_time: ", yupload_time)
                print("keywords: ", keywords)

                # meta tag data appending
                is_family_friendly_list.append(is_family_friendly)
                video_id_list.append(video_id)
                author_id_list.append(author_id)
                yupload_time_list.append(yupload_time)
                keywords_list.append(keywords)

                # ytInitialData crawling
                yt_initial_data = driver.find_element_by_xpath(yt_initial_data_xpath)
                yt_initial_data_str = yt_initial_data.get_attribute("innerHTML").split(";")[0].replace('window["ytInitialData"] =', '').strip()
                # print("yt_initial_data: ", yt_initial_data_str)

                time.sleep(5)

                # data to json
                # json loading이 간헐적으로 안되는 현상 수정 필요
                yt_initial_data_json = json.loads(yt_initial_data_str)

                # print("yt_initial_data_json: ", yt_initial_data_json)

                # video_info_renderer
                video_info_renderer = yt_initial_data_json['contents']['twoColumnWatchNextResults']['results']['results']['contents']
                video_primary_info_renderer = video_info_renderer[0]['videoPrimaryInfoRenderer']
                video_secondary_info_renderer = video_info_renderer[1]['videoSecondaryInfoRenderer']

                # print("video_primary_info_renderer: ", video_primary_info_renderer)
                # print("video_secondary_info_renderer: ", video_secondary_info_renderer)

                # 좋아요 & 싫어요
                like_dislike_cnt = video_primary_info_renderer['sentimentBar']['sentimentBarRenderer']['tooltip']

                print("like_dislike_cnt: ", like_dislike_cnt)

                like_cnt = int(like_dislike_cnt.split(" / ")[0].replace(",", ""))
                dislike_cnt = int(like_dislike_cnt.split(" / ")[1].replace(",", ""))

                print("like_cnt: ", like_cnt, " type: ", type(like_cnt))
                print("dislike_cnt: ", dislike_cnt, " type: ", type(dislike_cnt))

                # 제목 타이틀 해시태그
                # 타이틀 해시태그가 없는 경우 빈문자열로 처리
                raw_title_hash_tags = video_primary_info_renderer.get('superTitleLink', "")
                if raw_title_hash_tags != "":
                    title_hash_tags = video_primary_info_renderer['superTitleLink']['runs']
                    title_hash_tag_list = []
                    for tag in title_hash_tags:
                        if tag['text'] not in [" ", ""]:
                            title_hash_tag_list.append(tag['text'][1:])
                            # print("title_hash_tag: ", tag['text'][1:])
                    title_hash_tags_str = ",".join(title_hash_tag_list)
                else:
                    title_hash_tags_str = ""
                print("title_hash_tags_str: ", title_hash_tags_str)

                # 내용 해시태그
                description_hash_tags = video_secondary_info_renderer['description']['runs']
                description_hash_tag_list = []
                for tag in description_hash_tags:
                    if tag['text'] not in [" ", ""]:
                        if "#" in tag['text']:
                            description_hash_tag_list.append(tag['text'][1:])
                            # print("description_hash_tag: ", tag['text'][1:])

                description_hash_tags_str = ",".join(description_hash_tag_list)
                print("description_hash_tags_str: ", description_hash_tags_str)

                # 구독자 수
                # 구독자 수가 없는 경우에 대한 예외처리(KeyError) -> 0 으로 처리
                raw_follow = video_secondary_info_renderer['owner']['videoOwnerRenderer'].get('subscriberCountText', "")
                if raw_follow != "":
                    raw_follow_cnt = video_secondary_info_renderer['owner']['videoOwnerRenderer']['subscriberCountText']['runs'][0]['text'][4:]
                    follow_cnt = raw_follow_cnt[:len(raw_follow_cnt)-1]
                    print("follow_cnt: ", follow_cnt)

                    # 구독자 수 수치화 필터링
                    num_count_dict = {
                        '천' : 1000,
                        '만' : 10000,
                        '억' : 100000000,
                        '조' : 1000000000000
                    }

                    # print("follow_cnt: ", follow_cnt)
                    # print("follow_num_cnt: ", follow_cnt[-1])
                    # 천, 만, 억, 조 에 걸리는 경우 -> 그 만큼 곱해서 리턴, 그게 아닐 경우 그냥 정수화 후 리턴
                    if follow_cnt[-1] in num_count_dict.keys():
                        follow_cnt_num = int(float(follow_cnt[:len(follow_cnt)-1]) * num_count_dict[follow_cnt[-1]])
                    else:
                        follow_cnt_num = int(follow_cnt)
                else:
                    follow_cnt_num = 0
                    
                print("follow_cnt_num: ", follow_cnt_num, " type: ", type(follow_cnt_num))

                # yInitialData data appending
                like_cnt_list.append(like_cnt)
                dislike_cnt_list.append(dislike_cnt)
                title_hash_tags_list.append(title_hash_tags_str)
                description_hash_tags_list.append(description_hash_tags_str)
                follow_cnt_list.append(follow_cnt_num)
            
            except Exception as e:
                print()
                print("Exception: ", e)
                continue
            
            print("=" * 20, "Youtube Raw Data Descriptions Ended", "=" * 20, "\n")
            
            driver.close()

        # 얻어온 데이터를 dictionary에 저장
        youtube_raw_data['title'] = title_list
        youtube_raw_data['authorNick'] = author_list
        youtube_raw_data['description'] = description_list
        youtube_raw_data['urlPath'] = url_list
        youtube_raw_data['thumbUrl'] = thumb_url_list
        youtube_raw_data['likeCnt'] = like_cnt_list
        youtube_raw_data['dislikeCnt'] = dislike_cnt_list
        youtube_raw_data['followCnt'] = follow_cnt_list
        youtube_raw_data['viewCnt'] = view_cnt_list
        youtube_raw_data['duration'] = duration_list
        youtube_raw_data['utbCategory'] = youtube_category_list
        youtube_raw_data['utbCateSN'] = youtube_category_sn_list
        youtube_raw_data['isFamilyFriendly'] = is_family_friendly_list
        youtube_raw_data['titleHashTags'] = title_hash_tags_list
        youtube_raw_data['descHashTags'] = description_hash_tags_list
        youtube_raw_data['utbVideoID'] = video_id_list
        youtube_raw_data['authorID'] = author_id_list
        youtube_raw_data['yUploadDateTime'] = yupload_time_list
        youtube_raw_data['keywords'] = keywords_list

        # DataFrame 생성
        df = pd.DataFrame(youtube_raw_data)
        total_df.append(df, ignore_index = True)

        # 모아둔 DataFrame을 csv로 변환
        df.to_csv("./"+keyword+"_youtube_raw_data.csv")
    
    return total_df

In [77]:
scrap_search_detail_video_data(youtube_id_dict)


==================== Youtube Raw Data Descriptions ====================
title:  [11월 17일] 두산 vs NC 한국시리즈 1차전 경기 하이라이트 | 프로 야구 2020 NC Dinos vs Doosan Bears Highlights
author:  100야구큰 팬
description:  #두산vsNC, #NCvs두산,

[11월 17일] 두산 vs NC 한국시리즈 1차전 경기 하이라이트 | 프로 야구 2020 NC Dinos vs Doosan Bears Highlights
thumb_url:  https://i.ytimg.com/vi/doorlUyE9pI/hqdefault.jpg
view_cnt:  15672  type:  <class 'int'>
duration:  525  type:  <class 'int'>
youtube_category:  People & Blogs
youtube_category_id:  22
is_family_friendly:  True
video_id:  doorlUyE9pI
author_id:  UCma8ag4uAk_76d7wDHs5KqA
yupload_time:  2020-11-17
keywords:  
like_dislike_cnt:  113 / 9
like_cnt:  113  type:  <class 'int'>
dislike_cnt:  9  type:  <class 'int'>
title_hash_tags_str:  두산vsNC,NCvs두산
description_hash_tags_str:  두산vsNC,NCvs두산
follow_cnt_num:  0  type:  <class 'int'>
==================== Youtube Raw Data Descriptions Ended ==================== 


==================== Youtube Raw Data Descriptions ====================


like_dislike_cnt:  5,631 / 874
like_cnt:  5631  type:  <class 'int'>
dislike_cnt:  874  type:  <class 'int'>
title_hash_tags_str:  
description_hash_tags_str:  
follow_cnt:  1.83만
follow_cnt_num:  18300  type:  <class 'int'>
==================== Youtube Raw Data Descriptions Ended ==================== 


==================== Youtube Raw Data Descriptions ====================
title:  한국 프로야구 웃긴 영상모음
author:  온리원이글스
description:  KBO 프로야구 엠스플 웃긴 영상 모음3
웃긴 영상모음1 : https://youtu.be/Q103GIctf7U
웃긴 영상모음2 : https://youtu.be/N-fnIJXOR0I
thumb_url:  https://i.ytimg.com/vi/kH-ht4CyUKg/maxresdefault.jpg
view_cnt:  523813  type:  <class 'int'>
duration:  1046  type:  <class 'int'>
youtube_category:  Sports
youtube_category_id:  17
is_family_friendly:  True
video_id:  kH-ht4CyUKg
author_id:  UCgA_bbXQvYHXGQhOQEzPEkg
yupload_time:  2016-11-03
keywords:  KBO,웃긴 동영상,웃긴 야구동영상,웃긴 영상,웃긴 야구영상,프로야구 웃긴영상,프로야구 웃긴동영상,한국야구 웃긴동영상,한국야구 웃긴영상
like_dislike_cnt:  1,154 / 145
like_cnt:  1154  type:  <class 'int'>
dis


==================== Youtube Raw Data Descriptions ====================
title:  KBO 프로야구 다시는 나오기 힘든 기록들
author:  야구박사
description:  LEGEND
thumb_url:  https://i.ytimg.com/vi/wVo7-0zDNps/hqdefault.jpg
view_cnt:  838431  type:  <class 'int'>
duration:  541  type:  <class 'int'>
youtube_category:  Sports
youtube_category_id:  17
is_family_friendly:  True
video_id:  wVo7-0zDNps
author_id:  UC_xgQWu3RvW58p0_cofgfmA
yupload_time:  2020-03-04
keywords:  프로야구 KBO 기록

Exception:  Unterminated string starting at: line 1 column 16790 (char 16789)


ValueError: arrays must all be same length

In [70]:
# 데이터 단일 크롤링 테스트
def scrap_test_detail_video_data(video_id='y57Zldv2bik'):
    
    # webdriver 초기화
    driver_path = './chromedriver'
    driver = webdriver.Chrome(driver_path)
    driver.implicitly_wait(30) # or bigger second
    
    # origin_url 정의
    origin_url = 'https://www.youtube.com/watch?v='
    
    # 테스트용 단일 youtube_id 정의
    # (임시) 딘일로 넣어주는 값
    # video_id = 'y57Zldv2bik'
    
    # youtube_detail_raw_data를 받을 dictionary 정의
    youtube_raw_data = {}
    
    # 넣어주는 url 값
    url = origin_url + video_id
    
    # scriptTag에서 긁어지는 데이터
    script_tag_xpath = '/html/body/ytd-app/div/ytd-page-manager/ytd-watch-flexy/div[1]/ytd-player-microformat-renderer/script'
    
    # meta tag에서 긁어지는 데이터 -> soup.find, soup.findAll 이용해서 긁기
    request = requests.get(url)
    soup = BeautifulSoup(request.text, 'lxml')
    
    # ytInitialData에서 긁어지는 데이터 -> 복잡한 계층의 json 데이터 기반
    yt_initial_data_xpath = '/html/body/script[19]'
    
    # 수집할 데이터의 리스트
    title_list = []
    author_list = []
    description_list = []
    url_list = []
    thumb_url_list = []
    like_cnt_list = []
    dislike_cnt_list = []
    follow_cnt_list = []
    view_cnt_list = []
    duration_list = []
    youtube_category_list = []
    youtube_category_sn_list = []
    is_family_friendly_list = []
    title_hash_tags_list = []
    description_hash_tags_list = []
    video_id_list = [] 
    author_id_list = []
    yupload_time_list = []
    keywords_list = []
    
    driver.get(url)
    time.sleep(5)
    
    # Data 크롤링
    # 각 유형별로 다른 로직으로 데이터를 크롤링한다
    
    # script tag id=scriptTag의 내부 HTML 내용을 가져온다(scriptTag data crawling)
    script_tag_str = driver.find_element_by_xpath(script_tag_xpath).get_attribute("innerHTML")

    script_tag_json = json.loads(script_tag_str)
    
    time.sleep(5)

    # print("script_tag_json: ", script_tag_json)
    # print("script_tag_json_type: ", type(script_tag_json))

    # Data result test
    title = script_tag_json['name']
    author = script_tag_json['author']
    description = script_tag_json['description']
    thumb_url = script_tag_json['thumbnailUrl'][0]
    view_cnt = int(script_tag_json['interactionCount'])
    duration = int(script_tag_json['duration'][2:len(script_tag_json['duration'])-1].replace(",", ""))
    youtube_category = script_tag_json['genre']
    # addition : youtube_category_sn
    youtube_category_sn = youtube_category_dict[youtube_category]

    print("title: ", title)
    print("author: ", author)
    print("description: ", description)
    print("thumb_url: ", thumb_url)
    print("view_cnt: ", view_cnt, " type: ", type(view_cnt))
    print("duration: ", duration, " type: ", type(duration))
    print("youtube_category: ", youtube_category)
    print("youtube_category_id: ", youtube_category_sn)
    
    # scriptTag data appending
    title_list.append(title)
    author_list.append(author)
    description_list.append(description)
    url_list.append(url)
    thumb_url_list.append(thumb_url)
    view_cnt_list.append(view_cnt)
    duration_list.append(duration)
    youtube_category_list.append(youtube_category)
    youtube_category_sn_list.append(youtube_category_sn)
    
    
    # meta tag data crawling
    is_family_friendly = soup.find("meta", {"itemprop" : "isFamilyFriendly"})['content']
    author_id = soup.find("meta", {"itemprop" : "channelId"})['content']
    yupload_time = soup.find("meta", {"itemprop" : "uploadDate"})['content']
    keywords = ",".join([tag['content'] for tag in soup.find_all("meta", {"property" : "og:video:tag"})])

    print("is_family_friendly: ", is_family_friendly)
    print("video_id: ", video_id)
    print("author_id: ", author_id)
    print("yupload_time: ", yupload_time)
    print("keywords: ", keywords)
    
    # meta tag data appending
    is_family_friendly_list.append(is_family_friendly)
    video_id_list.append(video_id)
    author_id_list.append(author_id)
    yupload_time_list.append(yupload_time)
    keywords_list.append(keywords)
    
    # ytInitialData crawling
    yt_initial_data = driver.find_element_by_xpath(yt_initial_data_xpath)
    yt_initial_data_str = yt_initial_data.get_attribute("innerHTML").split(";")[0].replace('window["ytInitialData"] =', '').strip()
    # print("yt_initial_data: ", yt_initial_data_str)

    time.sleep(5)

    # data to json
    # json loading이 간헐적으로 안되는 현상 수정 필요
    yt_initial_data_json = json.loads(yt_initial_data_str)

    # print("yt_initial_data_json: ", yt_initial_data_json)

    # video_info_renderer
    video_info_renderer = yt_initial_data_json['contents']['twoColumnWatchNextResults']['results']['results']['contents']
    video_primary_info_renderer = video_info_renderer[0]['videoPrimaryInfoRenderer']
    video_secondary_info_renderer = video_info_renderer[1]['videoSecondaryInfoRenderer']

    # print("video_primary_info_renderer: ", video_primary_info_renderer)
    # print("video_secondary_info_renderer: ", video_secondary_info_renderer)

    # 좋아요 & 싫어요
    like_dislike_cnt = video_primary_info_renderer['sentimentBar']['sentimentBarRenderer']['tooltip']

    print("like_dislike_cnt: ", like_dislike_cnt)
    
    like_cnt = int(like_dislike_cnt.split(" / ")[0].replace(",", ""))
    dislike_cnt = int(like_dislike_cnt.split(" / ")[1].replace(",", ""))
    
    print("like_cnt: ", like_cnt, " type: ", type(like_cnt))
    print("dislike_cnt: ", dislike_cnt, " type: ", type(dislike_cnt))

    # 제목 타이틀 해시태그
    raw_title_hash_tags = video_primary_info_renderer.get('superTitleLink', "")
    if raw_title_hash_tags != "":
        title_hash_tags = video_primary_info_renderer['superTitleLink']['runs']
        title_hash_tag_list = []
        for tag in title_hash_tags:
            if tag['text'] not in [" ", ""]:
                title_hash_tag_list.append(tag['text'][1:])
                # print("title_hash_tag: ", tag['text'][1:])
        title_hash_tags_str = ",".join(title_hash_tag_list)
    else:
        title_hash_tags_str = raw_title_hash_tags
    print("title_hash_tags_str: ", title_hash_tags_str)

    # 내용 해시태그
    description_hash_tags = video_secondary_info_renderer['description']['runs']
    description_hash_tag_list = []
    for tag in description_hash_tags:
        if tag['text'] not in [" ", ""]:
            if "#" in tag['text']:
                description_hash_tag_list.append(tag['text'][1:])
                # print("description_hash_tag: ", tag['text'][1:])

    description_hash_tags_str = ",".join(description_hash_tag_list)
    print("description_hash_tags_str: ", description_hash_tags_str)

    # 구독자 수
    # 구독자 수가 없는 경우에 대한 예외처리(KeyError) -> 0 으로 처리
    raw_follow = video_secondary_info_renderer['owner']['videoOwnerRenderer'].get('subscriberCountText', "")
    if raw_follow != "":
        raw_follow_cnt = video_secondary_info_renderer['owner']['videoOwnerRenderer']['subscriberCountText']['runs'][0]['text'][4:]
        follow_cnt = raw_follow_cnt[:len(raw_follow_cnt)-1]
        print("follow_cnt: ", follow_cnt)

        # 구독자 수 수치화 필터링
        num_count_dict = {
            '천' : 1000,
            '만' : 10000,
            '억' : 100000000,
            '조' : 1000000000000
        }

        # print("follow_cnt: ", follow_cnt)
        # print("follow_num_cnt: ", follow_cnt[-1])
        # 천, 만, 억, 조 에 걸리는 경우 -> 그 만큼 곱해서 리턴, 그게 아닐 경우 그냥 정수화 후 리턴
        if follow_cnt[-1] in num_count_dict.keys():
            follow_cnt_num = int(float(follow_cnt[:len(follow_cnt)-1]) * num_count_dict[follow_cnt[-1]])
        else:
            follow_cnt_num = int(follow_cnt)
    else:
        follow_cnt_num = 0
    print("follow_cnt_num: ", follow_cnt_num, " type: ", type(follow_cnt_num))
    
    # yInitialData data appending
    like_cnt_list.append(like_cnt)
    dislike_cnt_list.append(dislike_cnt)
    title_hash_tags_list.append(title_hash_tags_str)
    description_hash_tags_list.append(description_hash_tags_str)
    follow_cnt_list.append(follow_cnt_num)
    
    driver.close()
    
    # 얻어온 데이터를 dictionary에 저장
    youtube_raw_data['title'] = title_list
    youtube_raw_data['authorNick'] = author_list
    youtube_raw_data['description'] = description_list
    youtube_raw_data['urlPath'] = url_list
    youtube_raw_data['thumbUrl'] = thumb_url_list
    youtube_raw_data['likeCnt'] = like_cnt_list
    youtube_raw_data['dislikeCnt'] = dislike_cnt_list
    youtube_raw_data['followCnt'] = follow_cnt_list
    youtube_raw_data['viewCnt'] = view_cnt_list
    youtube_raw_data['duration'] = duration_list
    youtube_raw_data['utbCategory'] = youtube_category_list
    youtube_raw_data['utbCateSN'] = youtube_category_sn_list
    youtube_raw_data['isFamilyFriendly'] = is_family_friendly_list
    youtube_raw_data['titleHashTags'] = title_hash_tags_list
    youtube_raw_data['descHashTags'] = description_hash_tags_list
    youtube_raw_data['utbVideoID'] = video_id_list
    youtube_raw_data['authorID'] = author_id_list
    youtube_raw_data['yUploadDateTime'] = yupload_time_list
    youtube_raw_data['keywords'] = keywords_list
    
    # DataFrame 생성
    df = pd.DataFrame(youtube_raw_data)
    
    return df

In [65]:
# 단일 크롤링 테스트 결과
scrap_test_detail_video_data(video_id='VjwJSj0AW4g')

title:  한국 프로야구 30주년 레전드 포수 이만수
author:  낙화유수
description:  설명
thumb_url:  https://i.ytimg.com/vi/VjwJSj0AW4g/hqdefault.jpg
view_cnt:  4992  type:  <class 'int'>
duration:  2999  type:  <class 'int'>
youtube_category:  People & Blogs
youtube_category_id:  22
is_family_friendly:  true
video_id:  VjwJSj0AW4g
author_id:  UCQm9eJus4FZKgJycDI7s9Mg
yupload_time:  2019-01-20
keywords:  
like_dislike_cnt:  33 / 1
like_cnt:  33  type:  <class 'int'>
dislike_cnt:  1  type:  <class 'int'>
title_hash_tags_str:  
description_hash_tags_str:  
follow_cnt:  842
follow_cnt_num:  842  type:  <class 'int'>


,title,authorNick,description,urlPath,thumbUrl,likeCnt,dislikeCnt,followCnt,viewCnt,duration,utbCategory,utbCateSN,isFamilyFriendly,titleHashTags,descHashTags,utbVideoID,authorID,yUploadDateTime,keywords
0,한국 프로야구 30주년 레전드 포수 이만수,낙화유수,설명,https://www.youtube.com/watch?v=VjwJSj0AW4g,https://i.ytimg.com/vi/VjwJSj0AW4g/hqdefault.jpg,33,1,842,4992,2999,People & Blogs,22,true,,,VjwJSj0AW4g,UCQm9eJus4FZKgJycDI7s9Mg,2019-01-20,


In [66]:
scrap_test_detail_video_data(video_id='doorlUyE9pI')

title:  [11월 17일] 두산 vs NC 한국시리즈 1차전 경기 하이라이트 | 프로 야구 2020 NC Dinos vs Doosan Bears Highlights
author:  100야구큰 팬
description:  #두산vsNC, #NCvs두산,

[11월 17일] 두산 vs NC 한국시리즈 1차전 경기 하이라이트 | 프로 야구 2020 NC Dinos vs Doosan Bears Highlights
thumb_url:  https://i.ytimg.com/vi/doorlUyE9pI/hqdefault.jpg
view_cnt:  15095  type:  <class 'int'>
duration:  525  type:  <class 'int'>
youtube_category:  People & Blogs
youtube_category_id:  22
is_family_friendly:  True
video_id:  doorlUyE9pI
author_id:  UCma8ag4uAk_76d7wDHs5KqA
yupload_time:  2020-11-17
keywords:  
like_dislike_cnt:  113 / 9
like_cnt:  113  type:  <class 'int'>
dislike_cnt:  9  type:  <class 'int'>
title_hash_tags_str:  두산vsNC,NCvs두산
description_hash_tags_str:  두산vsNC,NCvs두산
follow_cnt_num:  0  type:  <class 'int'>


,title,authorNick,description,urlPath,thumbUrl,likeCnt,dislikeCnt,followCnt,viewCnt,duration,utbCategory,utbCateSN,isFamilyFriendly,titleHashTags,descHashTags,utbVideoID,authorID,yUploadDateTime,keywords
0,[11월 17일] 두산 vs NC 한국시리즈 1차전 경기 하이라이트 | 프로 야구 ...,100야구큰 팬,"#두산vsNC, #NCvs두산,\n\n[11월 17일] 두산 vs NC 한국시리즈 ...",https://www.youtube.com/watch?v=doorlUyE9pI,https://i.ytimg.com/vi/doorlUyE9pI/hqdefault.jpg,113,9,0,15095,525,People & Blogs,22,True,"두산vsNC,NCvs두산","두산vsNC,NCvs두산",doorlUyE9pI,UCma8ag4uAk_76d7wDHs5KqA,2020-11-17,


In [71]:
scrap_test_detail_video_data(video_id='L0Y38pQbEeQ')

title:  한국 프로 야구(KBO 리그) 역대 한국 시리즈 우승팀 및 연도별 팀 순위 (1982-2018)
author:  데이터다람쥐
description:  1982년부터 2018년까지의 한국 프로야구 팀 연도별 순위 및 우승팀 입니다. 

음악:
Track: Miles Dominic - Lights Fading
Link: https://youtu.be/3PEFbYThi6U
thumb_url:  https://i.ytimg.com/vi/L0Y38pQbEeQ/maxresdefault.jpg
view_cnt:  245274  type:  <class 'int'>
duration:  292  type:  <class 'int'>
youtube_category:  Sports
youtube_category_id:  17
is_family_friendly:  True
video_id:  L0Y38pQbEeQ
author_id:  UCs-fnRl8JjhbMax7wRb9jwA
yupload_time:  2019-06-22
keywords:  야구,야구 순위,프로야구,프로 야구,한국 야구,한국 시리즈,한국시리즈,스포츠,그래프로 보는,한국 프로 야구,한국 프로야구,한화,기아,두산,삼성,sk,kt,위즈,넥센,키움,키움 히어로즈,두산 베어스,롯데 자이언츠,롯데,삼성 라이온즈,한화 이글스,기아 타이거즈,kt 위즈,kt wiz,lg 트윈스,엘지 트윈스
like_dislike_cnt:  1,386 / 73
like_cnt:  1386  type:  <class 'int'>
dislike_cnt:  73  type:  <class 'int'>
title_hash_tags_str:  
description_hash_tags_str:  
follow_cnt:  10.2만
follow_cnt_num:  102000  type:  <class 'int'>


,title,authorNick,description,urlPath,thumbUrl,likeCnt,dislikeCnt,followCnt,viewCnt,duration,utbCategory,utbCateSN,isFamilyFriendly,titleHashTags,descHashTags,utbVideoID,authorID,yUploadDateTime,keywords
0,한국 프로 야구(KBO 리그) 역대 한국 시리즈 우승팀 및 연도별 팀 순위 (198...,데이터다람쥐,1982년부터 2018년까지의 한국 프로야구 팀 연도별 순위 및 우승팀 입니다. \...,https://www.youtube.com/watch?v=L0Y38pQbEeQ,https://i.ytimg.com/vi/L0Y38pQbEeQ/maxresdefau...,1386,73,102000,245274,292,Sports,17,True,,,L0Y38pQbEeQ,UCs-fnRl8JjhbMax7wRb9jwA,2019-06-22,"야구,야구 순위,프로야구,프로 야구,한국 야구,한국 시리즈,한국시리즈,스포츠,그래프..."


In [78]:
scrap_test_detail_video_data(video_id='7eYu507yrQ8')

title:  이게 바로 눈호강 '슈퍼맨 캐치'!! KBO에서 나온 최고 레벨 호수비!! 타자가 탄식한다 "미친 거 아냐?..."
author:  엠빅뉴스
description:  삼성 외야수 박승규 선수가 MLB급 호수비를 연일 펼치고 있습니다. "보고도 믿을 수 없다"는 다이빙캐치를 거의 매경기 보여주고 있는데요. 이쯤에서 엠빅이 올시즌 나온 수비 명장면 모아봤습니다. 

#박승규 #KBO #박동원 #정수빈 #마차도 #다이빙캐치 #호수비 #슈퍼맨
thumb_url:  https://i.ytimg.com/vi/7eYu507yrQ8/maxresdefault.jpg
view_cnt:  1282593  type:  <class 'int'>
duration:  464  type:  <class 'int'>
youtube_category:  Pets & Animals
youtube_category_id:  15
is_family_friendly:  True
video_id:  7eYu507yrQ8
author_id:  UCYRrUD5v5j_Ei8sKOo7mhoQ
yupload_time:  2020-06-12
keywords:  박승규,KBO,박동원,정수빈,마차도,다이빙캐치,호수비,슈퍼맨


KeyError: 'videoPrimaryInfoRenderer'